In [18]:
# Code for Optimizing Supply Chain Cost under Trade-war
from gurobipy import *
import numpy as np

#########Parameters Set-up############

#Component landed cost by i: supplier, j: factory, k: component;  10000 are set for those Arcs not used.
RMCost = np.array([ [[71.5, 10000, 10000, 10000, 10000, 10000, 10000, 10000],
                     [74.78, 10000, 10000, 10000, 10000, 10000, 10000, 10000],
                     [72.1, 10000, 10000, 10000, 10000, 10000, 10000, 10000]],

                    [[74.58, 10000, 10000, 10000, 10000, 10000, 10000, 10000],
                     [71.7, 10000, 10000, 10000, 10000, 10000, 10000, 10000],
                     [75.18, 10000, 10000, 10000, 10000, 10000, 10000, 10000]],

                    [[10000, 23.69, 10000, 10000, 10000, 10000, 10000, 10000],
                     [10000, 22.85, 10000, 10000, 10000, 10000, 10000, 10000],
                     [10000, 22.45, 10000, 10000, 10000, 10000, 10000, 10000]],

                    [[10000, 22.85, 10000, 10000, 10000, 10000, 10000, 10000],
                     [10000, 23.69, 10000, 10000, 10000, 10000, 10000, 10000],
                     [10000, 24.09, 10000, 10000, 10000, 10000, 10000, 10000]],
                   
                    [[10000, 10000, 17.6, 10000, 10000, 10000, 10000, 10000],
                     [10000, 10000, 17.98, 10000, 10000, 10000, 10000, 10000],
                     [10000, 10000, 17.58, 10000, 10000, 10000, 10000, 10000]],
                   
                    [[10000, 10000, 17.98, 10000, 10000, 10000, 10000, 10000],
                     [10000, 10000, 17.6, 10000, 10000, 10000, 10000, 10000],
                     [10000, 10000, 17.2, 10000, 10000, 10000, 10000, 10000]],    
                   
                    [[10000, 10000, 10000, 43.86, 10000, 10000, 10000, 10000],
                     [10000, 10000, 10000, 41.35, 10000, 10000, 10000, 10000],
                     [10000, 10000, 10000, 44.26, 10000, 10000, 10000, 10000]],

                    [[10000, 10000, 10000, 41.35, 10000, 10000, 10000, 10000],
                     [10000, 10000, 10000, 43.86, 10000, 10000, 10000, 10000],
                     [10000, 10000, 10000, 40.95, 10000, 10000, 10000, 10000]],
                   
                    [[10000, 10000, 10000, 10000, 120.20, 10000, 10000, 10000],
                     [10000, 10000, 10000, 10000, 123.41, 10000, 10000, 10000],
                     [10000, 10000, 10000, 10000, 120.60, 10000, 10000, 10000]],                   

                    [[10000, 10000, 10000, 10000, 123.41, 10000, 10000, 10000],
                     [10000, 10000, 10000, 10000, 120.20, 10000, 10000, 10000],
                     [10000, 10000, 10000, 10000, 123.81, 10000, 10000, 10000]],
                   
                    [[10000, 10000, 10000, 10000, 10000, 38.67, 10000, 10000],
                     [10000, 10000, 10000, 10000, 10000, 37.80, 10000, 10000],
                     [10000, 10000, 10000, 10000, 10000, 38.20, 10000, 10000]],                   

                    [[10000, 10000, 10000, 10000, 10000, 37.8, 10000, 10000],
                     [10000, 10000, 10000, 10000, 10000, 38.67, 10000, 10000],
                     [10000, 10000, 10000, 10000, 10000, 39.07, 10000, 10000]],
                   
                    [[10000, 10000, 10000, 10000, 10000, 10000, 7.25, 10000],
                     [10000, 10000, 10000, 10000, 10000, 10000, 6.7, 10000],
                     [10000, 10000, 10000, 10000, 10000, 10000, 6.7, 10000]],                   

                    [[10000, 10000, 10000, 10000, 10000, 10000, 6.5, 10000],
                     [10000, 10000, 10000, 10000, 10000, 10000, 7.25, 10000],
                     [10000, 10000, 10000, 10000, 10000, 10000, 7.65, 10000]],
                   
                    [[10000, 10000, 10000, 10000, 10000, 10000, 10000, 30.60],
                     [10000, 10000, 10000, 10000, 10000, 10000, 10000, 30.60],
                     [10000, 10000, 10000, 10000, 10000, 10000, 10000, 30.66]],                   

                    [[10000, 10000, 10000, 10000, 10000, 10000, 10000, 31.06],
                     [10000, 10000, 10000, 10000, 10000, 10000, 10000, 31.06],
                     [10000, 10000, 10000, 10000, 10000, 10000, 10000, 30.20]]  ])

# Assembly cost/u for 3 factories.
AssemCost = ([24.5, 12, 26.5])

# Logistics cost of shipping 1 iphone from 3 factory to customers,
Logcost = ([2.9, 3.3, 1.8])

# I: Number of supplier 16, J: Number of factory 3, K: Number of component 8
I, J, K = RMCost.shape

# duty rate from factory 1 (China) to US customer
CN_US_duty_rate = 0.15

# duty rate for RM import to India factory
IN_RM_duty_rate = 0.1

#US market average demand unit per month in thousands.
demand = 1145

#Factory capacity for CN,IN,BR
capacity = ([800,360,120])

#iPhone Max XS selling price of 64G model
price = 1000

print(I)
print(J)
print(K)

16
3
8


In [19]:
# phase 1: minimize supply chain cost, 
# considering RMCost, RMcost only, Factory Assembly cost, CN to US duty rate, IN import duty rate

def model_setup():
    
    m = Model("SCCost")

    # Creat variables as Binary
    x = m.addVars(I, J, K, vtype=GRB.BINARY, name = "x")
    
    # set objective include: RMcost only, Factory Assembly cost, CN to US duty rate, IN import duty rate
    m.setObjective( quicksum( (((1+CN_US_duty_rate)* (RMCost[i,0,k] + AssemCost[0]/K)  )* x[i,0,k] 
                               + (RMCost[i, 1, k]*(1+IN_RM_duty_rate)+ AssemCost[1]/K )* x[i,1,k] 
                               + (RMCost[i, 2, k]+ AssemCost[2]/K ) * x[i,2,k]) for i in range(I) for k in range(K)), GRB.MINIMIZE)
    

    
    # There should be only 1 supplier for 1 component to 1 factory
    m.addConstrs( ( quicksum(x[i,j,k] for i in range(I)) == 1 for j in range(J) for k in range(K))  )
    
    # There can be max. 1 component supplied from 1 supplier to 1 factory
    m.addConstrs( ( quicksum(x[i,j,k] for k in range(K)) <= 1 for i in range(I) for j in range(J))  )
    
    #1 component can be supplied by 1 supplier to max of 3 countries (because we have 3 assemblers)
    m.addConstrs( ( quicksum(x[i,j,k] for j in range(J)) <= 3 for i in range(I) for k in range(K))  )    
    
    return m

In [20]:
# setup the model 
m_SC = model_setup()

# solving the model
m_SC.optimize()


# extract the variables from the model. NOTE: variables extracted in this way are automatically formatted as a vector
x = m_SC.getVars()

# reformat the vector as a matrix with dimension NxM
x = np.reshape(x, (I,J,K))

print("selected arcs = supplier, factory, component: RMcost")
for i in range(I):
    for j in range(J):
        for k in range(K):
            if x[i,j,k].x == 1:
                print(i+1, j+1, k+1, ":", RMCost[i,j,k]) 

# Store iphone cost of CN/IN/BR factory to US customer
FGCost = np.array([0.0,0.0,0.0])
FGCostwTariff = np.array([0.0,0.0,0.0])

for j in range(J):
    for k in range(K):
        for i in range(I):
            if x[i,j,k].x == 1: 
                FGCost[j] = FGCost[j] + RMCost[i,j,k]

# adding IN RM import duty
FGCost[1] = FGCost[1] * (1+IN_RM_duty_rate)

# adding Assembly cost
for j in range(J):
    FGCost[j] = FGCost[j] + AssemCost[j]
print("factory CN/IN/BR production cost(without CN-US duty):")
print(FGCost)

# adding CN_US duty
FGCostwTariff[0] = FGCost[0] * (1+CN_US_duty_rate)
FGCostwTariff[1] = FGCost[1]
FGCostwTariff[2] = FGCost[2]
print("factory CN/IN/BR production cost(include CN-US duty):")
print(FGCostwTariff)

Optimize a model with 200 rows, 384 columns and 1152 nonzeros
Variable types: 0 continuous, 384 integer (384 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 206353.16900
Presolve removed 176 rows and 256 columns
Presolve time: 0.00s
Presolved: 24 rows, 128 columns, 256 nonzeros
Found heuristic solution: objective 11173.395000
Variable types: 0 continuous, 128 integer (128 binary)

Root relaxation: objective 1.199415e+03, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1199.4150000 1199.41500  0.00%     -    0s

Explored 0 nodes (8 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 1199.42 11173.4 60940.7 

Optimal solution

In [26]:
#phase 2 - with the FG cost calculated from Phase 3, consider US customers demand and factory capacity, to maximize the profit

def model_setup():
    
    m = Model("SCProfit")

    #Creat variables y: quantity sold from each factory
    y = m.addVars(J, vtype=GRB.INTEGER, name = "y")
    
    # set objective for Max. Profit
    m.setObjective( quicksum(y[j]*(price - FGCostwTariff[j] - Logcost[j]) for j in range(J)), GRB.MAXIMIZE)

    # sum of each factory quantity should be <= demand
    m.addConstr( sum(y[j] for j in range(J)) <= demand)
    
    # each factory quantity should be <= capacity
    m.addConstr(y[0] <= capacity[0])
    m.addConstr(y[1] <= capacity[1])
    m.addConstr(y[2] <= capacity[2])

    m.addConstr(y[0] >= 0)
    m.addConstr(y[1] >= 0)    
    m.addConstr(y[2] >= 0)    
    
    return m


In [27]:
# setup the model 
m_SCProfit = model_setup()

# solving the model
m_SCProfit.optimize()

# extract the variables from the model. NOTE: variables extracted in this way are automatically formatted as a vector
y = m_SCProfit.getVars()

print("Quantity from factory CN/IN/BR:")
print(y[0].X, y[1].X, y[2].X)

Optimize a model with 7 rows, 3 columns and 9 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+03]
Found heuristic solution: objective 661963.90000
Presolve removed 7 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 669059 661964 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.690594250000e+05, best bound 6.690594250000e+05, gap 0.0000%
Quantity from factory CN/IN/BR:
665.0 360.0 120.0


In [28]:
#phase 2 benchmark model of maximize profit when there is no CN-US tariff

m_SCProfit.setObjective( quicksum(y[j]*(price - FGCost[j] - Logcost[j]) for j in range(J)), GRB.MAXIMIZE)

# solving the model
m_SCProfit.optimize()

# extract the variables from the model. NOTE: variables extracted in this way are automatically formatted as a vector
y = m_SCProfit.getVars()

print("Quantity from factory CN/IN/BR:")
print(y[0].X, y[1].X, y[2].X)

Optimize a model with 7 rows, 3 columns and 9 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+03]

Loaded MIP start with objective 706256

Presolve removed 7 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 709386 706256 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.093855000000e+05, best bound 7.093855000000e+05, gap 0.0000%
Quantity from factory CN/IN/BR:
800.0 225.0 120.0
